praktikum_d1

In [1]:
import cv2, time

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka. Coba index 1/2.")

frames, t0 = 0, time.time()
while True:
    ok, frame = cap.read()
    if not ok:
        break
    frames += 1
    if time.time() - t0 >= 1.0:
        cv2.setWindowTitle("Preview", f"Preview (FPS ~ {frames})")
        frames, t0 = 0, time.time()

    cv2.imshow("Preview", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


praktikum_d2

In [ ]:
import cv2, numpy as np
from cvzone.PoseModule import PoseDetector

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = PoseDetector(staticMode=False, modelComplexity=1,
                        enableSegmentation=False, detectionCon=0.5,
                        trackCon=0.5)
while True:
    # Tangkap setiap frame dari webcam
    success, img = cap.read()

    # Temukan pose manusia dalam frame
    img = detector.findPose(img)

    # Temukan landmark, bounding box, dan pusat tubuh dalam frame
    # Set draw=True untuk menggambar landmark dan bounding box pada
    # gambar
    lmList, bboxInfo = detector.findPosition(img, draw=True,
                                              bboxWithHands=False)

    # Periksa apakah ada landmark tubuh yang terdeteksi
    if lmList:
        # Dapatkan pusat bounding box di sekitar tubuh
        center = bboxInfo["center"]

        # Gambar lingkaran di pusat bounding box
        cv2.circle(img, center, 5, (255, 0, 255), cv2.FILLED)

        # Hitung jarak antara landmark 11 dan 15 dan gambarkan pada
        # gambar
        length, img, info = detector.findDistance(lmList[11][0:2],
                                                  lmList[15][0:2],
                                                  img=img,
                                                  color=(255, 0, 0),
                                                  scale=10)

        # Hitung sudut antara landmark 11, 13, dan 15 dan gambarkan
        # pada gambar
        angle, img = detector.findAngle(lmList[11][0:2],
                                        lmList[13][0:2],
                                        lmList[15][0:2],
                                        img=img,
                                        color=(0, 0, 255),
                                        scale=10)

        # Periksa apakah sudut mendekati 50 derajat dengan offset 10
        isCloseAngle50 = detector.angleCheck(myAngle=angle,
                                             targetAngle=50,
                                             offset=10)

        # Cetak hasil pemeriksaan sudut
        print(isCloseAngle50)

    cv2.imshow("Pose + Angle", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

praktikum_d3

In [3]:
import cv2, numpy as np
from cvzone.FaceMeshModule import FaceMeshDetector

# Indeks mata kiri (contoh: vertikal (159,145), horizontal (33,133)
L_TOP, L_BOTTOM, L_LEFT, L_RIGHT = 159, 145, 33, 133

def dist(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

# Inisialisasi objek FaceMeshDetector
# staticMode: Jika True, deteksi hanya terjadi sekali; jika False,
# setiap frame
# maxFaces: Jumlah maksimum wajah yang dideteksi
# minDetectionCon: Ambang kepercayaan deteksi minimum
# minTrackCon: Ambang kepercayaan pelacakan minimum
detector = FaceMeshDetector(staticMode=False, maxFaces=2,
                            minDetectionCon=0.5, minTrackCon=0.5)

# Variabel untuk menghitung kedipan sederhana
blink_count = 0
closed_frames = 0
CLOSED_FRAMES_THRESHOLD = 3 # jumlah frame berturut-turut untuk
                            # dianggap kedipan
EYE_AR_THRESHOLD = 0.20     # ambang Eye Aspect Ratio (EAR) untuk
                            # menilai mata tertutup
is_closed = False

while True:
    ok, img = cap.read()
    if not ok: break

    img, faces = detector.findFaceMesh(img, draw=True)

    if faces:
        face = faces[0]  # list of 468 (x,y)
        v = dist(face[L_TOP], face[L_BOTTOM])
        h = dist(face[L_LEFT], face[L_RIGHT])
        ear = v / (h + 1e-8)

        cv2.putText(img, f"EAR(L): {ear:.3f}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

        # Contoh ambang kedipan sederhana dan logika counter:
        # jika EAR < EYE_AR_THRESHOLD selama CLOSED_FRAMES_THRESHOLD
        # frame -> hitung kedipan
        if ear < EYE_AR_THRESHOLD:
            closed_frames += 1

            if closed_frames >= CLOSED_FRAMES_THRESHOLD and not is_closed:
                blink_count += 1
                is_closed = True
        else:
            closed_frames = 0
            is_closed = False

    # Tampilkan jumlah kedipan pada frame
    cv2.putText(img, f"Blink: {blink_count}", (20, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow("FaceMesh + EAR", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

praktikum_d4

In [4]:
import cv2
from cvzone.HandTrackingModule import HandDetector

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = HandDetector(staticMode=False, maxHands=1,
                        modelComplexity=1,
                        detectionCon=0.5, minTrackCon=0.5)

while True:
    ok, img = cap.read()
    if not ok: break

    hands, img = detector.findHands(img, draw=True, flipType=True) # flipType untuk mirror UI

    if hands:
        hand = hands[0] # dict berisi "lmList", "bbox", dll.
        fingers = detector.fingersUp(hand) # list panjang 5 berisi 0/1
        count = sum(fingers)
        cv2.putText(img, f"Fingers: {count}  {fingers}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow("Hands + Fingers", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

praktikum_d5

In [5]:
import cv2, numpy as np
from cvzone.HandTrackingModule import HandDetector

def dist(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

def classify_gesture(hand):
    # hand["lmList"] berisi 21 titik (x, y, z) dalam piksel saat
    # flipType=True
    lm = hand["lmList"]
    wrist = np.array(lm[0][0:2])
    thumb_tip = np.array(lm[4][0:2])
    index_tip = np.array(lm[8][0:2])
    middle_tip = np.array(lm[12][0:2])
    ring_tip = np.array(lm[16][0:2])
    pinky_tip = np.array(lm[20][0:2])

    # Heuristik jarak relatif
    r_mean = np.mean([dist(index_tip, wrist), dist(middle_tip, wrist),
                      dist(ring_tip, wrist), dist(pinky_tip, wrist),
                      dist(thumb_tip, wrist)])

    # Aturan:
    if dist(thumb_tip, index_tip) < 35:
        return "OK"

    # Thumbs up: ibu jari tinggi (y kecil), dan jauh dari wrist
    if (thumb_tip[1] < wrist[1] - 40) and (dist(thumb_tip, wrist) >
            0.8 * dist(index_tip, wrist)):
        return "THUMBS_UP"

    if r_mean < 120:
        return "ROCK"

    if r_mean > 200:
        return "PAPER"

    if (dist(index_tip, wrist) > 180 and dist(middle_tip, wrist) > 180 and
        dist(ring_tip, wrist) < 160 and dist(pinky_tip, wrist) < 160):
        return "SCISSORS"

    return "UNKNOWN"

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = HandDetector(staticMode=False, maxHands=1,
                        modelComplexity=1,
                        detectionCon=0.5, minTrackCon=0.5)

while True:
    ok, img = cap.read()
    if not ok: break

    hands, img = detector.findHands(img, draw=True, flipType=True)

    if hands:
        label = classify_gesture(hands[0])
        cv2.putText(img, f"Gesture: {label}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

    cv2.imshow("Hand Gestures (cvzone)", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

praktikum_d6

In [6]:
import cv2, numpy as np
from collections import deque
from cvzone.PoseModule import PoseDetector

MODE = "squat" # tekan 'm' untuk toggle ke "pushup"
KNEE_DOWN, KNEE_UP = 80, 160 # ambang squat (deg)
DOWN_R, UP_R = 0.85, 1.00 # ambang push-up (rasio)
SAMPLE_OK = 4 # minimal frame konsisten sebelum
              # ganti state

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = PoseDetector(staticMode=False, modelComplexity=1,
                        enableSegmentation=False, detectionCon=0.5,
                        trackCon=0.5)

count, state = 0, "up"
debounce = deque(maxlen=6)

def ratio_pushup(lm):
    # gunakan kiri: 11=shoulderL, 15=wristL, 23=hipL
    sh = np.array(lm[11][1:3])
    wr = np.array(lm[15][1:3])
    hp = np.array(lm[23][1:3])
    return np.linalg.norm(sh - wr) / (np.linalg.norm(sh - hp) + 1e-8)

while True:
    ok, img = cap.read()
    if not ok: break

    img = detector.findPose(img, draw=True)
    lmList, bboxInfo = detector.findPosition(img, draw=False) # [id, x, y, vis, ...]
    
    flag = None

    if lmList:
        if MODE == "squat":
            # Baris yang bermasalah telah dihapus. Nilai angL dan angR
            # akan didapatkan dari pemanggilan fungsi findAngle di bawah
            # yang juga berfungsi untuk menggambar.

            # Hitung dan Gambar Sudut Lutut Kiri
            angL, img = detector.findAngle(lmList[23][0:2],
                                          lmList[25][0:2],
                                          lmList[27][0:2],
                                          img=img,
                                          color=(0, 0, 255),
                                          scale=10)
            
            # Hitung dan Gambar Sudut Lutut Kanan
            angR, img = detector.findAngle(lmList[24][0:2],
                                          lmList[26][0:2],
                                          lmList[28][0:2],
                                          img=img,
                                          color=(0, 255, 0),
                                          scale=10)

            ang = (angL + angR) / 2.0
            cv2.putText(img, f"AvgKnee: {ang:.1f}", (20, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            
            if ang < KNEE_DOWN:
                flag = "down"
            elif ang > KNEE_UP:
                flag = "up"
                
        else: # MODE == "pushup"
            # push-up: rasio (shoulder-wrist) / (shoulder-hip)
            r = ratio_pushup(lmList)
            if r < DOWN_R:
                flag = "down"
            elif r > UP_R:
                flag = "up"
            cv2.putText(img, f"Ratio: {r:.4f}", (20, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        debounce.append(flag)

        if debounce.count("down") >= SAMPLE_OK and state == "up":
            state = "down"
        elif debounce.count("up") >= SAMPLE_OK and state == "down":
            count += 1
            state = "up"

    cv2.putText(img, f"Mode: {MODE.upper()} | Count: {count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
    cv2.putText(img, f"State: {state}", (20, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow("Pose Counter", img)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    if key == ord('m'): MODE = "pushup" if MODE == "squat" else "squat"

cap.release()
cv2.destroyAllWindows()